# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [14]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [15]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [16]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

#Get unique month values
df["issue_d"].unique()
df["next_pymnt_d"].unique()

months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5
}

#Get the proper month numbers
df["issue_m_num"] = df["issue_d"].apply(lambda x: months_num[x])
df["next_pymnt_m_num"] = df["next_pymnt_d"].apply(lambda x: months_num[x])

#Pick columns that can be dropped.
df["pymnt_plan"].unique() ##To be dropped
df["hardship_flag"].unique() ##To be dropped
df["debt_settlement_flag"].unique() ##To be dropped

df["initial_list_status"].unique() #1 of 2 can be dropped after encoding
df["application_type"].unique() #1 of 2 can be dropped after encoding


#Dummy encode
loans_binary_encoded = pd.get_dummies(df, columns=["home_ownership", 
                                                   "verification_status", 
                                                   "initial_list_status",
                                                   "application_type"])


cleaned_encoded_df = loans_binary_encoded.drop(["issue_d",
                                                "next_pymnt_d",
                                                "pymnt_plan",
                                                "hardship_flag",
                                                "debt_settlement_flag",
                                                "initial_list_status_f", 
                                                "application_type_Individual"], axis=1)

In [17]:
# Display the data
cleaned_encoded_df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,next_pymnt_m_num,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,5,0,0,0,1,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,5,0,1,0,0,0,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,5,0,1,0,0,0,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,5,0,0,0,1,0,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,5,0,1,0,0,1,0,0,1,0


## Split the Data into Training and Testing

In [18]:
# Create our features
X = cleaned_encoded_df.drop(columns="loan_status")

# Create our target
y = cleaned_encoded_df["loan_status"]

In [19]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,next_pymnt_m_num,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,4.616839,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,0.876121,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.486161,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.329446,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,5.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [21]:

# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=70)
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,next_pymnt_m_num,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App
13559,6525.0,0.2500,259.44,60000.0,8.90,0.0,0.0,7.0,0.0,1720.0,...,4,0,0,0,1,0,0,1,1,0
66744,16000.0,0.2437,463.74,59882.0,9.96,0.0,2.0,6.0,0.0,12317.0,...,5,0,1,0,0,0,1,0,1,0
14616,5000.0,0.1992,185.62,94500.0,37.02,1.0,0.0,12.0,0.0,48868.0,...,4,0,0,1,0,1,0,0,0,0
63093,14000.0,0.1180,463.67,65000.0,14.97,0.0,0.0,6.0,0.0,13079.0,...,5,0,0,0,1,1,0,0,1,0
58681,10000.0,0.0819,314.25,85000.0,15.54,0.0,1.0,12.0,0.0,39559.0,...,5,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27988,13200.0,0.1298,444.64,30000.0,27.64,1.0,0.0,18.0,0.0,12966.0,...,5,0,0,0,1,1,0,0,1,1
47733,10000.0,0.1033,214.10,96000.0,14.58,0.0,0.0,19.0,0.0,20198.0,...,4,0,1,0,0,1,0,0,1,0
45492,3000.0,0.1502,104.03,30000.0,30.68,0.0,0.0,7.0,1.0,3141.0,...,5,0,0,0,1,1,0,0,1,0
21563,16000.0,0.0819,325.88,54000.0,16.73,0.0,1.0,21.0,0.0,25954.0,...,4,0,1,0,0,0,1,0,1,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [22]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [23]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [24]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [26]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7478795620437956

In [27]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   46,    34],
       [ 1357, 15768]], dtype=int64)

In [28]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.57      0.92      0.06      0.73      0.51        80
   low_risk       1.00      0.92      0.57      0.96      0.73      0.55     17125

avg / total       0.99      0.92      0.58      0.95      0.73      0.55     17205



In [29]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_ 
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.0778393080607839, 'total_rec_prncp'),
 (0.06679440460545966, 'total_pymnt'),
 (0.06438469305413277, 'last_pymnt_amnt'),
 (0.0629446812711958, 'total_rec_int'),
 (0.05819745016811501, 'total_pymnt_inv'),
 (0.03266724484960699, 'int_rate'),
 (0.031006002666091294, 'issue_m_num'),
 (0.017921460012528232, 'max_bal_bc'),
 (0.01669139213983442, 'dti'),
 (0.016391956295420066, 'installment'),
 (0.016237691094134647, 'out_prncp'),
 (0.016109994055118492, 'mo_sin_old_rev_tl_op'),
 (0.015663213856700362, 'annual_inc'),
 (0.015505862162404866, 'mths_since_recent_inq'),
 (0.015218047278446585, 'tot_hi_cred_lim'),
 (0.014757738327305066, 'out_prncp_inv'),
 (0.014592423728850305, 'all_util'),
 (0.014563172983162068, 'il_util'),
 (0.014184478805530094, 'total_bc_limit'),
 (0.01415218213758878, 'mo_sin_old_il_acct'),
 (0.013858736576959846, 'tot_cur_bal'),
 (0.013783622529175319, 'avg_cur_bal'),
 (0.013643640758289178, 'revol_bal'),
 (0.013451981329695365, 'bc_util'),
 (0.013339005053987153, 'bc_o

### Easy Ensemble Classifier

In [30]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=10, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(random_state=1)

In [31]:

# Calculated the balanced accuracy score
y_pred = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9029872262773723

In [32]:

# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   69,    11],
       [  968, 16157]], dtype=int64)

In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.86      0.94      0.12      0.90      0.81        80
   low_risk       1.00      0.94      0.86      0.97      0.90      0.82     17125

avg / total       0.99      0.94      0.86      0.97      0.90      0.82     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier

2. Which model had the best recall score?

    The Easy Ensemble Classifier

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier

4. What are the top three features?

   last_pymnt_amnt, total_rec_prncp, total_pymnt